In [54]:
import mlflow

def get_metrics_and_params(run_id):
    client = mlflow.tracking.MlflowClient(tracking_uri='http://127.0.0.1:5000/')
    run = client.get_run(run_id)
    metrics = run.data.metrics
    params = run.data.params
    return metrics, params

def get_mean_and_std(val1, val2, val3):
    mean = (val1 + val2 + val3) / 3
    std = ((val1 - mean)**2 + (val2 - mean)**2 + (val3 - mean)**2) / 3
    return round(mean,2), round(std,2)

def get_metrics_overview(retrained_metrics_1, retrained_metrics_2, retrained_metrics_3, unlearned_metrics_1, unlearned_metrics_2, unlearned_metrics_3, percentage=True):
    if 'MIA_prob' not in retrained_metrics_1:
        retrained_metrics_1['MIA_prob'] = retrained_metrics_1['membership_attack_prob']
    if percentage:
        for met in ["MIA_prob", "acc_forget", "acc_retain"]:
            retrained_metrics_1[met] = round(retrained_metrics_1[met]/100,2)
            retrained_metrics_2[met] = round(retrained_metrics_2[met]/100,2)
            retrained_metrics_3[met] = round(retrained_metrics_3[met]/100,2)


    r_mia_mean, r_mia_std = get_mean_and_std(retrained_metrics_1['MIA_prob'], retrained_metrics_2['MIA_prob'], retrained_metrics_3['MIA_prob'])
    r_acc_forget_mean, r_acc_forget_std = get_mean_and_std(retrained_metrics_1['acc_forget'], retrained_metrics_2['acc_forget'], retrained_metrics_3['acc_forget'])
    r_acc_retain_mean, r_acc_retain_std = get_mean_and_std(retrained_metrics_1['acc_retain'], retrained_metrics_2['acc_retain'], retrained_metrics_3['acc_retain'])
    r_t_mean, r_t_std = get_mean_and_std(retrained_metrics_1['t'], retrained_metrics_2['t'], retrained_metrics_3['t'])
    r_l2_mean, r_l2_std = get_mean_and_std(retrained_metrics_1['l2'], retrained_metrics_2['l2'], retrained_metrics_3['l2'])

    u_mia_mean, u_mia_std = get_mean_and_std(unlearned_metrics_1['MIA_prob'], unlearned_metrics_2['MIA_prob'], unlearned_metrics_3['MIA_prob'])
    u_acc_forget_mean, u_acc_forget_std = get_mean_and_std(unlearned_metrics_1['acc_forget'], unlearned_metrics_2['acc_forget'], unlearned_metrics_3['acc_forget'])
    u_acc_retain_mean, u_acc_retain_std = get_mean_and_std(unlearned_metrics_1['acc_retain'], unlearned_metrics_2['acc_retain'], unlearned_metrics_3['acc_retain'])
    u_t_mean, u_t_std = get_mean_and_std(unlearned_metrics_1['t'], unlearned_metrics_2['t'], unlearned_metrics_3['t'])
    u_l2_mean, u_l2_std = get_mean_and_std(unlearned_metrics_1['l2'], unlearned_metrics_2['l2'], unlearned_metrics_3['l2'])
    u_ve_mean, u_ve_std = get_mean_and_std(unlearned_metrics_1['VE'], unlearned_metrics_2['VE'], unlearned_metrics_3['VE'])

    diff_acc_forget_mean, diff_acc_forget_std = get_mean_and_std(abs(retrained_metrics_1['acc_forget'] - unlearned_metrics_1['acc_forget']), abs(retrained_metrics_2['acc_forget'] - unlearned_metrics_2['acc_forget']), abs(retrained_metrics_3['acc_forget'] - unlearned_metrics_3['acc_forget']))
    diff_acc_retain_mean, diff_acc_retain_std = get_mean_and_std(abs(retrained_metrics_1['acc_retain'] - unlearned_metrics_1['acc_retain']), abs(retrained_metrics_2['acc_retain'] - unlearned_metrics_2['acc_retain']), abs(retrained_metrics_3['acc_retain'] - unlearned_metrics_3['acc_retain']))

    retrained_metrics_overview = {"MIA": f"{r_mia_mean} ± {r_mia_std}", "acc_forget": f"{r_acc_forget_mean} ± {r_acc_forget_std}", "acc_retain": f"{r_acc_retain_mean} ± {r_acc_retain_std}", "t": f"{r_t_mean} ± {r_t_std}", "l2": f"{r_l2_mean} ± {r_l2_std}"}
    unlearned_metrics_overview = {"MIA": f"{u_mia_mean} ± {u_mia_std}", "acc_forget": f"{u_acc_forget_mean} ± {u_acc_forget_std} ({diff_acc_forget_mean} ± {diff_acc_forget_std})", "acc_retain": f"{u_acc_retain_mean} ± {u_acc_retain_std} ({diff_acc_retain_mean} ± {diff_acc_retain_std})", "t": f"{u_t_mean} ± {u_t_std}", "l2": f"{u_l2_mean} ± {u_l2_std}", "ve": f"{u_ve_mean} ± {u_ve_std}"}
    return retrained_metrics_overview, unlearned_metrics_overview

# Instance-wise Unlearning
Description: Forget specific datapoints across all the classes.

## CIFAR-10 (ResNet18)

In [55]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("4933a16e6e594a849a80a1aebb0132a8")
retrained_metrics2, retrained_params2 = get_metrics_and_params("115c419853bd4ca5b7812bb0a39411bb")
retrained_metrics3, retrained_params3 = get_metrics_and_params("6f1f93b6b21c44c5b232f92819931885")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("8b8049e1778747b983b1ec4f8cc7847e")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("59f2c21a24ea4ceaaf93b32e27b45400")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("d6b39048de914a3682419ba542aad400")

# Hyper-parameters of unlearned models
print(f"Hyperparams: {unlearned_params1}, {unlearned_params2}, {unlearned_params3}")

cifar10_resnet_retrained_metrics, cifar10_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3)
print(f"Retrained: {cifar10_resnet_retrained_metrics}")
print(f"Unlearned: {cifar10_resnet_unlearned_metrics}")

Hyperparams: {'reference_run_name': 'retrained', 'reference_run_id': '4933a16e6e594a849a80a1aebb0132a8', 'seed': '3407', 'cudnn': 'slow', 'dataset': 'cifar-10', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'minTemp': '1.1', 'maxTemp': '16.0', 'probTransformer': 'gumbel-softmax', 'datetime': '09-09-16-53', 'epochs': '10', 'method': 'our', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'model': 'resnet18', 'batch_size': '128', 'beta': '0'}, {'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'probTransformer': 'gumbel-softmax', 'beta': '0', 'reference_run_name': 'retrained', 'seed': '13', 'cudnn': 'slow', 'model': 'resnet18', 'method': 'ou

## CIFAR-100 (ResNet18)

In [56]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("d2d5d47fee764f8aa3ef70f464e8fc71")
retrained_metrics2, retrained_params2 = get_metrics_and_params("807e0d22e0714b6e85f08d2f4c40c1f4")
retrained_metrics3, retrained_params3 = get_metrics_and_params("080fcf72b43a40c1b44f6a4d472b4b26")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("fec0002f10214d4fa849d3508d3fc18e")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("61aaf4e9a4134d268b10e313fafe7a08")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("a82abae8634f4246b060c89c29b73cee")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

cifar100_resnet_retrained_metrics, cifar100_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3)
print(f"Retrained: {cifar100_resnet_retrained_metrics}")
print(f"Unlearned: {cifar100_resnet_unlearned_metrics}")

{'seed': '3407', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'maxTemp': '4.0', 'beta': '0', 'cudnn': 'slow', 'dataset': 'cifar-100', 'model': 'resnet18', 'epochs': '10', 'method': 'our', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential', 'datetime': '09-09-17-16', 'reference_run_name': 'retrained', 'reference_run_id': 'd2d5d47fee764f8aa3ef70f464e8fc71', 'minTemp': '1.1', 'probTransformer': 'gumbel-softmax', 'batch_size': '128'} {'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'wd': '0.0005', 'tempScheduler': 'exponential', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'lr': '0.0001', 'optimizer': 'Adam (\nParameter Group 0\n    am

## MUFAC (ResNet18)

In [58]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("7fbc281913c74e6aa1eef899758b9ff2")
retrained_metrics2, retrained_params2 = get_metrics_and_params("cb0ce77530b84dc193593a6d340ae978")
retrained_metrics3, retrained_params3 = get_metrics_and_params("28a660ce59e5477da488052c62d717cc")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("5ef737a644314bacbc5ccc492c5fb9e3")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("92d2fd7c6ee44ab58d01f10e1974a3a8")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("c31fcbc172b44dcd87de5cdf646c572d")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

mufac_resnet_retrained_metrics, mufac_resnet_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=False)
print(f"Retrained: {mufac_resnet_retrained_metrics}")
print(f"Unlearned: {mufac_resnet_unlearned_metrics}")

{'datetime': '09-09-17-51', 'reference_run_id': '7fbc281913c74e6aa1eef899758b9ff2', 'cudnn': 'slow', 'model': 'resnet18', 'minTemp': '1.1', 'maxTemp': '32.0', 'probTransformer': 'gumbel-softmax', 'beta': '0', 'reference_run_name': 'retrained', 'seed': '3407', 'dataset': 'mufac', 'batch_size': '128', 'epochs': '10', 'method': 'our', 'git_commit_hash': '61852c1aeb3f58d8f2dbb925a42ce9e530b9cdef', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'wd': '0.0005', 'optimizer': 'Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    capturable: False\n    differentiable: False\n    eps: 1e-08\n    foreach: None\n    fused: None\n    lr: 0.0001\n    maximize: False\n    weight_decay: 0.0005\n)', 'tempScheduler': 'exponential'} {'method': 'our', 'branch_name': 'annealing', 'Dr_subset_size': '0.3', 'lr': '0.0001', 'epochs': '10', 'tempScheduler': 'exponential', 'maxTemp': '32.0', 'probTransformer': 'gumbel-softmax', 'beta': '0', 'wd': '0.0005', 'optimizer'

## IageNet (ResNet18)

## CIFAR-10 (ViT)

In [ ]:
retrained_metrics1, retrained_params1 = get_metrics_and_params("")
retrained_metrics2, retrained_params2 = get_metrics_and_params("")
retrained_metrics3, retrained_params3 = get_metrics_and_params("")

unlearned_metrics1, unlearned_params1 = get_metrics_and_params("")
unlearned_metrics2, unlearned_params2 = get_metrics_and_params("")
unlearned_metrics3, unlearned_params3 = get_metrics_and_params("")

# Hyper-parameters of unlearned models
print(unlearned_params1, unlearned_params2, unlearned_params3)

cifar10_vit_retrained_metrics, cifar10_vit_unlearned_metrics = get_metrics_overview(retrained_metrics1, retrained_metrics2, retrained_metrics3, unlearned_metrics1, unlearned_metrics2, unlearned_metrics3, percentage=False)
print(f"Retrained: {cifar10_vit_retrained_metrics}")
print(f"Unlearned: {cifar10_vit_unlearned_metrics}")